In [5]:
import os
import argparse
import logging
import urllib
from tqdm import tqdm
import sys
import codecs
import spacy
import torch
import tarfile
import torchtext.data
import torchtext.datasets


In [6]:
import transformers
transformers.__version__

'4.38.2'

In [43]:
x = torch.randn(4, 4)
x

tensor([[-0.2170, -0.4064, -0.2708,  0.1597],
        [-1.8654,  0.7103, -0.5406,  0.0239],
        [ 1.0485,  0.0319,  1.0114, -0.8338],
        [-0.7670, -0.2378,  1.1530, -0.5154]])

In [44]:
x.view((-1, 8)).shape, x.view(-1).shape, x.view(8).shape

RuntimeError: shape '[8]' is invalid for input of size 16

In [14]:
inds = torch.tensor([1, 3, 4])
x.index_select(1, inds)

RuntimeError: INDICES element is out of DATA bounds, id=4 axis_dim=4

In [17]:
l = torch.nn.Conv1d(3, 5, 2)
l.weight, l.bias

(Parameter containing:
 tensor([[[ 0.0593,  0.2881],
          [-0.1172,  0.3856],
          [ 0.3556,  0.1064]],
 
         [[ 0.0139,  0.3622],
          [-0.3303, -0.3028],
          [-0.2213, -0.2269]],
 
         [[-0.3487, -0.0465],
          [ 0.2822, -0.0413],
          [-0.1009, -0.0081]],
 
         [[ 0.1015, -0.1141],
          [-0.1849, -0.2004],
          [-0.2604, -0.3764]],
 
         [[ 0.0411,  0.3473],
          [-0.2646,  0.2422],
          [-0.2797, -0.2754]]], requires_grad=True),
 Parameter containing:
 tensor([0.1624, 0.2841, 0.3526, 0.2781, 0.2666], requires_grad=True))

In [70]:
l.bias.index_select(0, torch.tensor([1, 2, 7]))

IndexError: index out of range in self

In [19]:
l.weight.size()

torch.Size([5, 3, 2])

In [23]:
torch.full((), 77) == torch.tensor(77)

tensor(True)

In [25]:
x = torch.arange(20)
x

tensor([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
        18, 19])

In [32]:
x[-5:5]

tensor([], dtype=torch.int64)

In [35]:
t = torch.rand((5, 5))
t

tensor([[0.1698, 0.6858, 0.8964, 0.8091, 0.9632],
        [0.6114, 0.9482, 0.2565, 0.2160, 0.9927],
        [0.7067, 0.4991, 0.5201, 0.0048, 0.8184],
        [0.6011, 0.7674, 0.0509, 0.1306, 0.8765],
        [0.4807, 0.2753, 0.3314, 0.7934, 0.7542]])

In [37]:
t1 = torch.tril(t)
t1

tensor([[0.1698, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.6114, 0.9482, 0.0000, 0.0000, 0.0000],
        [0.7067, 0.4991, 0.5201, 0.0000, 0.0000],
        [0.6011, 0.7674, 0.0509, 0.1306, 0.0000],
        [0.4807, 0.2753, 0.3314, 0.7934, 0.7542]])

In [38]:
t1[:3, :4]

tensor([[0.1698, 0.0000, 0.0000, 0.0000],
        [0.6114, 0.9482, 0.0000, 0.0000],
        [0.7067, 0.4991, 0.5201, 0.0000]])

In [45]:
t.topk(2)

torch.return_types.topk(
values=tensor([[0.9632, 0.8964],
        [0.9927, 0.9482],
        [0.8184, 0.7067],
        [0.8765, 0.7674],
        [0.7934, 0.7542]]),
indices=tensor([[4, 2],
        [4, 1],
        [4, 0],
        [4, 1],
        [3, 4]]))

In [49]:
t = torch.rand((3, 5))
t

tensor([[0.0074, 0.9776, 0.0541, 0.8086, 0.1039],
        [0.6737, 0.5848, 0.8657, 0.7908, 0.4491],
        [0.9662, 0.0886, 0.9440, 0.5317, 0.6208]])

In [54]:
sz = 3
sz = 1
m = torch.tril(torch.ones((sz, sz), dtype=torch.bool))
m

tensor([[True]])

In [56]:
t1 = t.clone()
t1 = t1.masked_fill(m, -1)
t1

tensor([[-1., -1., -1., -1., -1.],
        [-1., -1., -1., -1., -1.],
        [-1., -1., -1., -1., -1.]])

In [65]:
t

tensor([[0.0074, 0.9776, 0.0541, 0.8086, 0.1039],
        [0.6737, 0.5848, 0.8657, 0.7908, 0.4491],
        [0.9662, 0.0886, 0.9440, 0.5317, 0.6208]])

In [68]:
inds = torch.tensor([0, 2, 1])
t[inds, :2]

tensor([[0.0074, 0.9776],
        [0.9662, 0.0886],
        [0.6737, 0.5848]])

In [71]:
a = torch.arange(10).reshape(5, 2)
a

tensor([[0, 1],
        [2, 3],
        [4, 5],
        [6, 7],
        [8, 9]])

In [75]:
a.split((2, 3))

(tensor([[0, 1],
         [2, 3]]),
 tensor([[4, 5],
         [6, 7],
         [8, 9]]))

In [76]:
t = torch.ones((2, 3, 4))
t

tensor([[[1., 1., 1., 1.],
         [1., 1., 1., 1.],
         [1., 1., 1., 1.]],

        [[1., 1., 1., 1.],
         [1., 1., 1., 1.],
         [1., 1., 1., 1.]]])

In [79]:
c = torch.nn.Conv1d(3, 2, 1)
c.weight, c.bias

(Parameter containing:
 tensor([[[ 0.3082],
          [-0.5674],
          [ 0.0346]],
 
         [[-0.3134],
          [ 0.4890],
          [ 0.1158]]], requires_grad=True),
 Parameter containing:
 tensor([-0.5212, -0.4785], requires_grad=True))

In [80]:
c(t)

tensor([[[-0.7457, -0.7457, -0.7457, -0.7457],
         [-0.1871, -0.1871, -0.1871, -0.1871]],

        [[-0.7457, -0.7457, -0.7457, -0.7457],
         [-0.1871, -0.1871, -0.1871, -0.1871]]],
       grad_fn=<ConvolutionBackward0>)

In [81]:
c((t, t))


TypeError: conv1d() received an invalid combination of arguments - got (tuple, Parameter, Parameter, tuple, tuple, tuple, int), but expected one of:
 * (Tensor input, Tensor weight, Tensor bias, tuple of ints stride, tuple of ints padding, tuple of ints dilation, int groups)
      didn't match because some of the arguments have invalid types: (!tuple of (Tensor, Tensor)!, !Parameter!, !Parameter!, !tuple of (int,)!, !tuple of (int,)!, !tuple of (int,)!, int)
 * (Tensor input, Tensor weight, Tensor bias, tuple of ints stride, str padding, tuple of ints dilation, int groups)
      didn't match because some of the arguments have invalid types: (!tuple of (Tensor, Tensor)!, !Parameter!, !Parameter!, !tuple of (int,)!, !tuple of (int,)!, !tuple of (int,)!, int)


In [82]:
c.forward(t)

tensor([[[-0.7457, -0.7457, -0.7457, -0.7457],
         [-0.1871, -0.1871, -0.1871, -0.1871]],

        [[-0.7457, -0.7457, -0.7457, -0.7457],
         [-0.1871, -0.1871, -0.1871, -0.1871]]],
       grad_fn=<ConvolutionBackward0>)

In [3]:
import os
from datetime import datetime
import gzip
from pathlib import Path
import re

DT_PAT = '%Y%m%d_%H%M%S'
DT_PAT_RE = r'\d{8}_\d{6}'

s = 'encdec-20240718_221554-wiki_20200501_en-ch_100_fixed'

subdir_pat = r'^\w+\-(%s)-.+$' % DT_PAT_RE
print(subdir_pat)
pat = re.compile(r'^\w+\-(%s)-.+$' % DT_PAT_RE)
m = pat.match(s)
if m:
    print(m)
    dt_str = m.group(1)
    print(dt_str)
    dt = datetime.strptime(dt_str, DT_PAT)
    print(dt)


^\w+\-(\d{8}_\d{6})-.+$
<re.Match object; span=(0, 52), match='encdec-20240718_221554-wiki_20200501_en-ch_100_fi>
20240718_221554
2024-07-18 22:15:54


In [5]:
fpath = Path(os.path.expandvars('$HOME')) / 'Downloads' / 'msmarco-doctrain-top100.gz'
with gzip.open(fpath, 'rt', encoding='utf8') as f:
    for i in range(105):
        l = f.readline()
        print(l.rstrip())

1185869 Q0 D59221 1 -4.80433 IndriQueryLikelihood
1185869 Q0 D59220 2 -4.92127 IndriQueryLikelihood
1185869 Q0 D2192591 3 -5.05215 IndriQueryLikelihood
1185869 Q0 D2777518 4 -5.05486 IndriQueryLikelihood
1185869 Q0 D2371978 5 -5.07048 IndriQueryLikelihood
1185869 Q0 D1186567 6 -5.09707 IndriQueryLikelihood
1185869 Q0 D2510055 7 -5.1502 IndriQueryLikelihood
1185869 Q0 D59219 8 -5.15332 IndriQueryLikelihood
1185869 Q0 D1371943 9 -5.2028 IndriQueryLikelihood
1185869 Q0 D1379540 10 -5.21803 IndriQueryLikelihood
1185869 Q0 D2305872 11 -5.30554 IndriQueryLikelihood
1185869 Q0 D899138 12 -5.31173 IndriQueryLikelihood
1185869 Q0 D2422342 13 -5.3206 IndriQueryLikelihood
1185869 Q0 D59228 14 -5.32493 IndriQueryLikelihood
1185869 Q0 D2022152 15 -5.3397 IndriQueryLikelihood
1185869 Q0 D262064 16 -5.34528 IndriQueryLikelihood
1185869 Q0 D2089121 17 -5.36559 IndriQueryLikelihood
1185869 Q0 D1964065 18 -5.37196 IndriQueryLikelihood
1185869 Q0 D1611531 19 -5.38115 IndriQueryLikelihood
1185869 Q0 D2286

[2, 1, 2]
